# Un peu d'Astropy et d'astroQuery



In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt

# Téléchargement d'une image d'exemple
from astropy.utils.data import download_file
image_file = download_file('https://astropy.stsci.edu/data/tutorials/FITS-images/HorseHead.fits', cache=True)

# Ouverture du fichier FITS
hdul = fits.open(image_file)
hdul.info()

# Visualisation de l’image
image_data = hdul[0].data
plt.imshow(image_data, cmap='gray')
plt.colorbar()
plt.title("Image FITS — Tête de Cheval")
plt.show()

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

# Créer une coordonnée équatoriale (ICRS)
coord = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')
print("Coordonnée ICRS :", coord)

# Conversion en galactique
print("Coordonnée galactique :", coord.galactic)

In [ ]:
from astropy.table import QTable
import astropy.units as u

t = QTable()
t['source'] = ['M1', 'M42', 'M81']
t['ra'] = [83.633, 83.822, 148.888] * u.deg
t['dec'] = [22.0145, -5.3911, 69.0653] * u.deg
t['mag'] = [8.4, 4.0, 6.9]

from astropy.coordinates import SkyCoord
t['coord'] = SkyCoord(ra=t['ra'], dec=t['dec'], frame='icrs')

t.pprint()

In [ ]:
from astropy.time import Time

t = Time("2024-01-01T00:00:00")
print("Date UTC :", t.utc.iso)
print("Julian Date :", t.jd)
print("Temps sidéral à Greenwich :", t.sidereal_time('mean', 'greenwich'))

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)

wcs = WCS(hdul[0].header)

fig = plt.figure(figsize=(6, 6))
ax = plt.subplot(projection=wcs)
ax.imshow(image_data, origin='lower', cmap='gray')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
plt.title("Image avec WCS")
plt.show()

 ### Exemple de calcul de séparation angulaire

In [ ]:
c1 = SkyCoord(ra=10*u.deg, dec=20*u.deg, frame='icrs')
c2 = SkyCoord(ra=11*u.deg, dec=21*u.deg, frame='icrs')
sep = c1.separation(c2)
print("Séparation angulaire :", sep)

In [ ]:
from astropy.io import fits
from specutils import Spectrum1D
import matplotlib.pyplot as plt
import numpy as np

# URL d’un spectre de quasar (SDSS)
url = "https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/1320/spZbest-1320-52759.fits"

# Chargement du fichier FITS en ligne
hdulist = fits.open(url)
hdulist.info()


hdul = fits.open(url)
hdul.info()

# Afficher les noms de colonnes de l'extension 1 ou 2
for ext in [1, 2]:
    try:
        print(f"\nExtension {ext} columns:", hdul[ext].data.columns.names)
    except Exception:
        pass

In [ ]:
from astroquery.simbad import Simbad

result = Simbad.query_object("M1")
print(result)

print("RA/DEC :", result['ra'][0], result['dec'][0])

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

coord = SkyCoord("05h34m31.94s +22d00m52.2s", frame="icrs")
result = Simbad.query_region(coord, radius="5m")
print(result)

## 2. VizieR — Requêter des catalogues d'étoiles, galaxies, etc.

🔹 Chercher un catalogue spécifique par mot-clé

In [ ]:
from astroquery.vizier import Vizier

Vizier.ROW_LIMIT = 10  # limiter le nombre de lignes
catalogs = Vizier.find_catalogs("gaia")
print(catalogs.keys())

🔹 Extraire des données Gaia autour d’un objet

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

coord = SkyCoord.from_name("M1")
viz = Vizier(columns=["RAJ2000", "DEJ2000", "Gmag"], row_limit=50)
result = viz.query_region(coord, radius=5 * u.arcmin, catalog="I/345/gaia2")
table = result[0]
table.pprint()

## 🌌 3. SkyView — Télécharger des images depuis des grands surveys

### 🔹 Télécharger une image FITS de M51 depuis DSS2

In [ ]:
from astroquery.skyview import SkyView

images = SkyView.get_images(position="M51", survey="DSS2 Red")

from astropy.io import fits
import matplotlib.pyplot as plt

data = images[0][0].data
plt.imshow(data, cmap='gray', origin='lower')
plt.title("M51 - DSS2 Red")
plt.colorbar()
plt.show()

In [28]:
ra1 = 150.1   # en degrés
ra2 = 150.4
dec1 = 2.0
dec2 = 2.3

from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord
import astropy.units as u

# Définir les coins en RA/DEC
ra_center = (ra1 + ra2) / 2
dec_center = (dec1 + dec2) / 2
width = abs(ra2 - ra1) * u.deg
height = abs(dec2 - dec1) * u.deg

coord = SkyCoord(ra=ra_center * u.deg, dec=dec_center * u.deg, frame='icrs')

# Télécharger l'image depuis un survey (ex: DSS, 2MASS, GALEX, etc.)
images = SkyView.get_images(position=coord, survey='DSS2 Red',
                            width=width, height=height)

# Visualisation
from astropy.io import fits
import matplotlib.pyplot as plt

data = images[0][0].data
plt.imshow(data, origin='lower', cmap='gray')
plt.title("Image extraite via SkyView")
plt.colorbar()
plt.show()


KeyboardInterrupt: 